# Dynamic Manning Report - Doctor Data

VADM Burke, Chief of Naval Personnel, ordered LTJGs Dannelly, Calcagno, Shaw, and Kuzma to build a prototype of a job-seeker / job-owner matching algorithm in one month to aid personnel detailing. This report offers an overview of the team’s results utilizing medical corps data provided by CDR Gingo. 

In [1]:
#Data Ingestion
#Import Libraries
import pandas as pd
import pref_metrics as pm

#Path to files
file_path_data = '/Users/z/Documents/Data_Cleaning/med_data/clean/'
file_path_results = '/Users/z/Documents/Data_Cleaning/med_data/clean/results/'

#Raw Data
s_df = pd.read_csv(file_path_data + "S.csv", index_col=0)
o_df = pd.read_csv(file_path_data + "O.csv", index_col=0)
a_df = pd.read_csv(file_path_data + "A.csv", index_col=0)
x_tn_df = pd.read_csv(file_path_data + "X.csv", index_col=0)


#Match Dataframes
x_MIP_df = pd.read_csv(file_path_results + "X_mip.csv", index_col=0)
x_da_s_df = pd.read_csv(file_path_results + "X_da_s.csv", index_col=0)
x_da_o_df = pd.read_csv(file_path_results + "X_da_o.csv", index_col=0)

#Similarity Dataframes
sim_s_df = pd.read_csv(file_path_results + "Similarity_s.csv", index_col=0)
sim_o_df = pd.read_csv(file_path_results + "Similarity_o.csv", index_col=0)

#Post Match Stats
post_stat_df = pd.read_csv(file_path_results + "post_match.csv",index_col=0)


#Metrics Dataframes
metrics_s_df = pd.read_csv(file_path_results + "Metrics_s.csv", index_col=0)
metrics_s_df['Pref'] = (x_MIP_df * s_df).sum(axis=0)

metrics_o_df = pd.read_csv(file_path_results + "Metrics_o.csv", index_col=0)
metrics_s_df['Pref'] = (x_MIP_df.T * o_df).sum(axis=0)

#Correlation Dataframes
corr_s_df = pd.read_csv(file_path_results + "Corr_s.csv", index_col=0)
corr_o_df = pd.read_csv(file_path_results + "Corr_o.csv", index_col=0)

#Air Force Status Quo Metrics
af_metrics = pd.DataFrame([.21, .42, .55], index=["Assigned 1st Pref", "Assigned Top 5 Pref", "Assigned Top 10 Pref"])

#Add in totals
total_1 = (post_stat_df.T["1_o_count"] + post_stat_df.T["1_s_count"])
total_5 = (post_stat_df.T["5_o_count"] + post_stat_df.T["5_s_count"]).rename("Ratio 5")
total_10 = (post_stat_df.T["10_o_count"] + post_stat_df.T["10_s_count"]).rename("Ratio 10")
total_participants = post_stat_df.T["o_participants"] + post_stat_df.T["s_participants"]

ratio_1 = (total_1 / total_participants).rename("Total 1 Ratio")
ratio_5 = (total_5 / total_participants).rename("Total 5 Ratio")
ratio_10 = (total_10 / total_participants).rename("Total 10 Ratio")


post_stat_df = post_stat_df.append([ratio_1,ratio_5,ratio_10])

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

plot_metrics = post_stat_df.iloc[[50, 51, 52], :]

plot_metrics.index = ["Assigned 1st Pref", "Assigned Top 5 Pref", "Assigned Top 10 Pref"]

data = [
    go.Bar(
        x=af_metrics.index, # assign x as the dataframe column 'x'
        y=af_metrics[0],
        name = "Detailing Standard"
    ),
    go.Bar(
        x=plot_metrics.index, # assign x as the dataframe column 'x'
        y=plot_metrics["da_s"],
        name = "Differed Acceptance"
    ),
    go.Bar(
        x=plot_metrics.index, # assign x as the dataframe column 'x'
        y=plot_metrics["mip"],
        name = "Mixed Integer Algo"
    )
]

layout = go.Layout(
    barmode='group',
    title='Overall Preference Allocation',
    yaxis=dict(
        tickformat='%d'
    )
)

fig = go.Figure(data=data, layout=layout)

# IPython notebook
py.iplot(fig, filename='pandas-bar-chart-layout')

In [ ]:
fig = {
    'data': [
      {
      'x': metrics_s_df.Specialization, 
            'y': metrics_s_df.Competitiveness, 
            'text': metrics_s_df.index, 
            'mode': 'markers', 
            'name': 'Sailors'}
    ],
    'layout': {
        'xaxis': {'title': 'Spec', 'type': 'log'},
        'yaxis': {'title': "Comp"}
    }
}

# IPython notebook
py.iplot(fig, filename='out')

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = {
  "data": [
    {
      "values": [16, 15, 12],
      "labels": [
        "first",
        "top5",
        "rest"
      ],
      'domain': {'x': [0, .5], 'y': [.52, 1]},
      "name": "1",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Global Emissions 1990-2011",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "DA-O",
                "x": 0.22,
                "y": 0.20
            }
        ]
    }
}
py.iplot(fig, filename='donut')

In [ ]:
O_comp = {}
for key, value in match_dict.items():
    total_comp = 0
    for item in value:
        total_comp += metrics_s_df.at[item,"Competitiveness"]
    O_comp[key] = total_comp
O_comp

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = {
  "data": [
    {
      "values": list(O_comp.values()),
      "labels": list(O_comp.keys()),
      "name": "1",
      "hoverinfo":"label+percent+name",
      "type": "pie"
    }],
  "layout": {
        "title":"Talent Allocation: Cumulative Competitiveness "
    }
}
py.iplot(fig, filename='donut')

In [ ]:
lista = list(post_stat_df.index)
lista.index("Total 1 Ratio")

In [ ]:
(x_da_s_df * s_df).sum().sum()

In [ ]:
(x_tn_df * s_df).sum().sum()

In [ ]:
(x_tn_df.T * o_df).sum().sum()

In [ ]:
(x_da_s_df.T * o_df).sum().sum()

In [ ]:
post_stat_df

In [ ]:
import qgrid
widget = qgrid.show_grid(metrics_s_df, show_toolbar=True)
widget

In [ ]:
import qgrid
widget = qgrid.show_grid(metrics_o_df, show_toolbar=True)
widget

In [ ]:
spec = (s_df.kurtosis(axis=1) * s_df.skew(axis=1)) * -1
spec_norm = (spec - spec.min()) / (spec.max() - spec.min())
spec_norm.to_dict()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
s_df.T['command_A'].value_counts().sort_index().plot(ax=ax, kind='bar')
fig

In [ ]:
pd.set_option('display.max_columns', len(s_df.columns))
o_df

In [2]:
metrics_o_df['Pref'] = (pd.DataFrame(x_MIP_df.as_matrix() * o_df.as_matrix().T, index=x_MIP_df.index, columns=x_MIP_df.columns).sum(axis=0)).divide(a_df)
metrics_o_df

ValueError: cannot copy sequence with size 60 to array axis with dimension 20

In [3]:
(pd.DataFrame(x_MIP_df.as_matrix() * o_df.as_matrix().T, index=x_MIP_df.index, columns=x_MIP_df.columns).sum(axis=1))
#.divide(a_df['0'])
#(x_MIP_df * o_df.T).sum(axis=1)/a_df['0']

command_A      1.0
command_B      8.0
command_C     38.0
command_D     12.0
command_E    107.0
command_F     51.0
command_G      1.0
command_H     55.0
command_I      3.0
command_J     53.0
command_K     44.0
command_L     21.0
command_M     24.0
command_N     69.0
command_O     92.0
command_P     48.0
command_Q     62.0
command_R      3.0
command_S     35.0
command_T     20.0
dtype: float64

In [6]:
import numpy as np

In [13]:
pd.DataFrame(x_MIP_df.as_matrix() * o_df.as_matrix().T)

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
(x_MIP_df * o_df.T) - pd.DataFrame(x_MIP_df.as_matrix() * o_df.as_matrix().T, index=x_MIP_df.index, columns=x_MIP_df.columns)



,officer_01,officer_02,officer_03,officer_04,officer_05,officer_06,officer_07,officer_08,officer_09,officer_10,...,officer_50,officer_51,officer_52,officer_53,officer_54,officer_55,officer_56,officer_57,officer_58,officer_59
command_A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-20.0,0.0,0.0,0.0,0.0
command_D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0
command_F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
pd.DataFrame(x_MIP_df.as_matrix() * o_df.as_matrix().T, index=x_MIP_df.index, columns=x_MIP_df.columns)

,officer_01,officer_02,officer_03,officer_04,officer_05,officer_06,officer_07,officer_08,officer_09,officer_10,...,officer_50,officer_51,officer_52,officer_53,officer_54,officer_55,officer_56,officer_57,officer_58,officer_59
command_A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0
command_D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.0,0.0,0.0
command_F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_H,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_I,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
(x_MIP_df * o_df.T)

,officer_01,officer_02,officer_03,officer_04,officer_05,officer_06,officer_07,officer_08,officer_09,officer_10,...,officer_50,officer_51,officer_52,officer_53,officer_54,officer_55,officer_56,officer_57,officer_58,officer_59
command_A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0
command_D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0
command_F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_H,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_I,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
command_J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
o_df

,command_A,command_B,command_C,command_D,command_E,command_F,command_G,command_H,command_I,command_J,command_K,command_L,command_M,command_N,command_O,command_P,command_Q,command_R,command_S,command_T
officer_01,19.0,26.0,19.0,19.0,24.0,19.0,19.0,24.0,19.0,20.0,2.0,28.0,27.0,25.0,22.0,2.0,6.0,20.0,24.0,21.0
officer_02,18.0,25.0,18.0,18.0,23.0,18.0,18.0,1.0,18.0,3.0,19.0,26.0,26.0,23.0,20.0,20.0,2.0,19.0,23.0,20.0
officer_03,54.0,54.0,54.0,54.0,52.0,53.0,52.0,55.0,55.0,55.0,54.0,12.0,54.0,55.0,55.0,54.0,55.0,55.0,55.0,55.0
officer_04,34.0,9.0,34.0,35.0,37.0,35.0,35.0,34.0,33.0,34.0,35.0,40.0,40.0,3.0,34.0,35.0,37.0,35.0,36.0,36.0
officer_05,25.0,13.0,25.0,25.0,29.0,25.0,25.0,10.0,25.0,26.0,25.0,32.0,33.0,30.0,27.0,27.0,31.0,25.0,5.0,27.0
officer_06,23.0,29.0,23.0,23.0,28.0,23.0,23.0,27.0,23.0,24.0,22.0,10.0,30.0,29.0,2.0,25.0,5.0,23.0,27.0,25.0
officer_07,11.0,22.0,10.0,11.0,15.0,4.0,4.0,17.0,3.0,8.0,12.0,20.0,22.0,17.0,15.0,13.0,23.0,12.0,15.0,15.0
officer_08,45.0,47.0,45.0,46.0,47.0,45.0,45.0,44.0,45.0,47.0,46.0,50.0,48.0,2.0,41.0,45.0,44.0,47.0,45.0,48.0
officer_10,30.0,32.0,29.0,30.0,31.0,27.0,27.0,31.0,27.0,2.0,28.0,37.0,38.0,35.0,32.0,7.0,33.0,27.0,33.0,30.0
officer_11,44.0,49.0,44.0,43.0,2.0,43.0,44.0,47.0,46.0,45.0,44.0,46.0,49.0,48.0,46.0,44.0,46.0,46.0,47.0,46.0


In [ ]:
x_MIP_df

In [20]:
metrics_s_df

,Competitiveness,Specialization,Pref
officer_01,0.9280,0.054199,NaN
officer_02,0.9311,0.053823,NaN
officer_03,0.8785,0.961797,NaN
officer_04,0.9051,0.230170,NaN
officer_05,0.9174,0.075156,NaN
officer_06,0.9221,0.078100,NaN
officer_07,0.9396,0.028226,NaN
officer_08,0.8899,0.869050,NaN
officer_09,0.8713,0.034937,NaN
officer_10,0.9118,0.150320,NaN


In [22]:
o_df

,command_A,command_B,command_C,command_D,command_E,command_F,command_G,command_H,command_I,command_J,command_K,command_L,command_M,command_N,command_O,command_P,command_Q,command_R,command_S,command_T
officer_01,19.0,26.0,19.0,19.0,24.0,19.0,19.0,24.0,19.0,20.0,2.0,28.0,27.0,25.0,22.0,2.0,6.0,20.0,24.0,21.0
officer_02,18.0,25.0,18.0,18.0,23.0,18.0,18.0,1.0,18.0,3.0,19.0,26.0,26.0,23.0,20.0,20.0,2.0,19.0,23.0,20.0
officer_03,54.0,54.0,54.0,54.0,52.0,53.0,52.0,55.0,55.0,55.0,54.0,12.0,54.0,55.0,55.0,54.0,55.0,55.0,55.0,55.0
officer_04,34.0,9.0,34.0,35.0,37.0,35.0,35.0,34.0,33.0,34.0,35.0,40.0,40.0,3.0,34.0,35.0,37.0,35.0,36.0,36.0
officer_05,25.0,13.0,25.0,25.0,29.0,25.0,25.0,10.0,25.0,26.0,25.0,32.0,33.0,30.0,27.0,27.0,31.0,25.0,5.0,27.0
officer_06,23.0,29.0,23.0,23.0,28.0,23.0,23.0,27.0,23.0,24.0,22.0,10.0,30.0,29.0,2.0,25.0,5.0,23.0,27.0,25.0
officer_07,11.0,22.0,10.0,11.0,15.0,4.0,4.0,17.0,3.0,8.0,12.0,20.0,22.0,17.0,15.0,13.0,23.0,12.0,15.0,15.0
officer_08,45.0,47.0,45.0,46.0,47.0,45.0,45.0,44.0,45.0,47.0,46.0,50.0,48.0,2.0,41.0,45.0,44.0,47.0,45.0,48.0
officer_10,30.0,32.0,29.0,30.0,31.0,27.0,27.0,31.0,27.0,2.0,28.0,37.0,38.0,35.0,32.0,7.0,33.0,27.0,33.0,30.0
officer_11,44.0,49.0,44.0,43.0,2.0,43.0,44.0,47.0,46.0,45.0,44.0,46.0,49.0,48.0,46.0,44.0,46.0,46.0,47.0,46.0


In [28]:
o_df.sort_index(axis=0).sort_index(axis=1,inplace=True)

In [29]:
o_df

,command_A,command_B,command_C,command_D,command_E,command_F,command_G,command_H,command_I,command_J,command_K,command_L,command_M,command_N,command_O,command_P,command_Q,command_R,command_S,command_T
officer_01,19.0,26.0,19.0,19.0,24.0,19.0,19.0,24.0,19.0,20.0,2.0,28.0,27.0,25.0,22.0,2.0,6.0,20.0,24.0,21.0
officer_02,18.0,25.0,18.0,18.0,23.0,18.0,18.0,1.0,18.0,3.0,19.0,26.0,26.0,23.0,20.0,20.0,2.0,19.0,23.0,20.0
officer_03,54.0,54.0,54.0,54.0,52.0,53.0,52.0,55.0,55.0,55.0,54.0,12.0,54.0,55.0,55.0,54.0,55.0,55.0,55.0,55.0
officer_04,34.0,9.0,34.0,35.0,37.0,35.0,35.0,34.0,33.0,34.0,35.0,40.0,40.0,3.0,34.0,35.0,37.0,35.0,36.0,36.0
officer_05,25.0,13.0,25.0,25.0,29.0,25.0,25.0,10.0,25.0,26.0,25.0,32.0,33.0,30.0,27.0,27.0,31.0,25.0,5.0,27.0
officer_06,23.0,29.0,23.0,23.0,28.0,23.0,23.0,27.0,23.0,24.0,22.0,10.0,30.0,29.0,2.0,25.0,5.0,23.0,27.0,25.0
officer_07,11.0,22.0,10.0,11.0,15.0,4.0,4.0,17.0,3.0,8.0,12.0,20.0,22.0,17.0,15.0,13.0,23.0,12.0,15.0,15.0
officer_08,45.0,47.0,45.0,46.0,47.0,45.0,45.0,44.0,45.0,47.0,46.0,50.0,48.0,2.0,41.0,45.0,44.0,47.0,45.0,48.0
officer_09,57.0,57.0,57.0,57.0,59.0,59.0,57.0,57.0,59.0,58.0,58.0,57.0,58.0,57.0,58.0,57.0,58.0,58.0,57.0,58.0
officer_10,30.0,32.0,29.0,30.0,31.0,27.0,27.0,31.0,27.0,2.0,28.0,37.0,38.0,35.0,32.0,7.0,33.0,27.0,33.0,30.0
